# iDARTS 

Unlike some recent papers that have proposed regularization approaches for the DARTS, the iDARTS {cite:p}`zhang2021idarts` proposes a different way to compute the outer-loop gradient based on the implicit function theorem.<br> 

The Amended-DARTS {cite:p}`bi2020stabilizing` has actually arrived at the same outer-loop gradient (hypergradient) formula, stating that the gradient approximation in the original DARTS has been inaccurate. {cite:p}`bi2020stabilizing` have replaced the inverse Hessian with the Hessian itself in the formula to make computation more tractable however calculating the Hessian is still costly and it takes more time for the Amended-DARTS to reach optimality. Replacing the inverse Hessian within Hessian is also not a good approximation.

The iDARTS on the other hand, approximates the inverse Hessian and thus reduces the computational cost of the Amended-DARTS, as well as provides better results.

From the Original DARTS we had: <br>
$\nabla_\alpha L_{val} (w^*(a),a) \approx \nabla_\alpha L_{val} (w - \xi \nabla_w L_{train}(w,\alpha),a)$ <br>

Ideally, we'd want the $w - \xi \nabla_w L_{train}(w,\alpha)$ to be as close to the optimal $w^*(\alpha)$ as possible however this is very unlikely because as a result of the approximation the weights of the inner loop get updated only once.


However, if indeed $w ' = w - \xi \nabla_w L_{train}(w,\alpha)$ was optimal then we would have the training loss gradient $\nabla_{w} L_{train}(w^*,\alpha) = 0$ {cite:p}`zhang2021idarts`

Thus:<br>
$ \nabla^2_{w,\alpha} L_{train}(w^*,\alpha) = 0$<br>
$ \nabla_{w} (\nabla_{\alpha} L_{train}(w^*,\alpha)) = 0$<br>
$ \nabla_{w} (\nabla_{\alpha} L_{train}(w^*,\alpha) + \nabla_{w} L_{train}(w^*,\alpha). \nabla_{\alpha} w) = 0$<br>
$ \nabla^2_{w,\alpha} L_{train}(w^*,\alpha) + \nabla^2_{w,w} L_{train}(w^*,\alpha). \nabla_{\alpha} w = 0$<br>
$\nabla_{\alpha} w = - \big[\nabla^2_{w,w} L_{train}(w^*,\alpha)   \big]^{-1} . \nabla^2_{w,\alpha} L_{train}(w^*,\alpha)$<br>

Because the inverse Hessian of the training loss w.r.t the weights is expensive to compute the authors have proposed to use the Neumann approximation approach {cite:p}`DBLP:journals/corr/abs-1911-02590`
computed based on mini-batch samples. 

$\big[\nabla^2_{w,w} L_{train}(w^*,\alpha)   \big]^{-1} = \underset{i \to \infty}{lim} \overset{i}{\underset{j=0}{\sum}} \big[ I - \nabla^2_{w,w} L_{train}(w^*,\alpha) \big]^j    $

By using only the first K terms of the Neumann series:<br>
$\nabla_\alpha L_{val} (w',a) \approx \nabla_\alpha L_{val} (w' ,a) - \xi \nabla^2_{w\alpha} L_{train}(w,\alpha). \overset{K}{\underset{j=0}{\sum}} \big[ I - \xi \nabla^2_{w,w} L_{train}(w^*,\alpha) \big]^j  . \nabla_{w'} L_{val}(w',\alpha) \ \ \ \ $<br>

In [1]:
import numpy as np
from numpy.linalg import inv
from numpy import linalg as LA

In [2]:
N = 3
a = np.random.random((N,N))
b = inv(a)
np.max(LA.eig(a)[0]),b

(1.459229411118642,
 array([[-0.46990415, -3.70471549,  8.11896613],
        [-9.30718464,  8.10625168, -0.77413482],
        [ 7.28290496, -4.27425816, -1.43084164]]))

In [3]:
N = 3
a = 100*np.random.random((N,N))
b = inv(a)
np.max(LA.eig(a)[0]),b

((126.3174746413699+0j),
 array([[-0.0170664 ,  0.05880986, -0.06530811],
        [ 0.00715687, -0.02650302,  0.04658296],
        [ 0.01668402,  0.00207029, -0.01405238]]))

In [4]:
N = 3
a = 10000*np.random.random((N,N))
b = inv(a)
np.max(LA.eig(a)[0]),b

(14231.355537709951,
 array([[ 2.16246117e-03, -9.75949516e-04, -4.06264704e-04],
        [-3.45531582e-04,  2.78670084e-04,  2.80231176e-05],
        [-7.62258628e-04,  3.09354492e-04,  2.56525350e-04]]))

The larger the Hessian Norm, the smaller the inverse Hessian, so smaller step in that direction.

Determinant is product of eigenvalues https://math.berkeley.edu/~chenhi/math54-u15/problem150714.pdf.<br>
inv(A) = 1/det(A) . adj(A)<br>

Large eigenvalues (sharpness) > large determinant > smaller inverse matrix 